Importing libraries

In [ ]:
import sqlite3
import pandas as pd
import pycytominer

In [ ]:
#Here paste the path where is located the .db file
inputDir = "/Users/llanos/Documents/GitHub/Kyra_project/Batch4"
#name of the .df file without the extension
database_name='DS2U_DS1_Neurons_03042024_database'

SQLITE FILE PROCESSING


In [ ]:
# Path of the SQLite database
database_path = f"{inputDir}/{database_name}.db"

# Establish a connection to the SQLite database
connection = sqlite3.connect(database_path)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Query to retrieve table names from sqlite_master
table_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
cursor.execute(table_query)

# Add a new column named "Metadata_genotype" to the existing table
cursor.execute('''ALTER TABLE MyExpt_Per_Image
                  ADD COLUMN Metadata_genotype TEXT;''')

# Update the values in the new column based on the conditions, for row B and C and E and F

cursor.execute('''UPDATE MyExpt_Per_Image
                  SET Metadata_genotype = CASE 
                                       WHEN Image_Metadata_Well LIKE '%B%' OR Image_Metadata_Well LIKE '%C%' THEN 'DS2U'
                                       WHEN Image_Metadata_Well LIKE '%E%' OR Image_Metadata_Well LIKE '%F%' THEN 'DS1'
                                       ELSE NULL  -- You can change this to a default value if needed
                                       END;''')


# Fetch all table names
table_names = cursor.fetchall()


# Rename the table in a format that can be read in the pipeline
old_table_Image = 'MyExpt_Per_Image'
new_table_Image = 'Image'
old_table_Nuclei = 'MyExpt_Per_Nuclei'
new_table_Nuclei = 'Nuclei'
old_table_Cytoplasm = 'MyExpt_Per_Cytoplasm'
new_table_Cytoplasm = 'Cytoplasm'
old_table_Cells = 'MyExpt_Per_Cells'
new_table_Cells = 'Cells'

# Use the ALTER TABLE statement to rename the table
cursor.execute(f"ALTER TABLE {old_table_Image} RENAME TO {new_table_Image}")
cursor.execute(f"ALTER TABLE {old_table_Nuclei} RENAME TO {new_table_Nuclei}")
cursor.execute(f"ALTER TABLE {old_table_Cytoplasm} RENAME TO {new_table_Cytoplasm}")
cursor.execute(f"ALTER TABLE {old_table_Cells} RENAME TO {new_table_Cells}")


#Add a new column for Image_Metadata_Table

alter_query = '''
ALTER TABLE Image
ADD COLUMN Image_Metadata_Plate TEXT;
'''

cursor.execute(alter_query)

# Update all existing rows in the Image_Metadata_Table with the string "plate1"
update_query = '''
UPDATE Image
SET Image_Metadata_Plate = 'plate1';
'''

cursor.execute(update_query)


# Commit the changes
connection.commit()


# Close the cursor and database connection
cursor.close()
connection.close()



OperationalError: no such table: MyExpt_Per_Image

Read the tables name in the database file

In [ ]:
# Display the table names inside of the sqlite files
for table_name in table_names:
    print(table_name[0])

In [ ]:
#MAKE A NEW SQLITE FILE WITH THE FOUR TABLES: Cells, Nuclei, Cytoplasm and image THAT ARE READ FOR THE PROFILING RECEIPE

def copy_tables(source_db, destination_db, tables):
    with sqlite3.connect(source_db) as source_conn:
        with source_conn as source_cursor:
            # Attach the destination database
            source_cursor.execute(f"ATTACH DATABASE '{destination_db}' AS dest")

            # Loop through the specified tables and copy them to the destination
            for table in tables:
                source_cursor.execute(f"CREATE TABLE dest.{table} AS SELECT * FROM {table}")

# Replace these values with your actual database file names and table names
source_database = f"{inputDir}/{database_name}.db"
destination_database = f"{inputDir}/{database_name}.sqlite"
tables_to_copy = ['Cells', 'Nuclei', 'Cytoplasm','Image']

# Call the function to copy tables
copy_tables(source_database, destination_database, tables_to_copy)

GROUPING VALUES FROM EACH TABLE FROM SQLITE FILE BASED ON IMAGENUMBER COLUMN

In [ ]:
#Aggregating values from each Image in Cells table

connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Cells"
data_frame_objects = pd.read_sql(query, connection)
data_frame_objects.to_csv(f"{inputDir}/Cells.csv", index=False) 

grouped_df = data_frame_objects.groupby('ImageNumber')

# Applying aggregate functions (e.g., sum, mean) to each group
result_df_Cells = grouped_df.mean().reset_index(drop=False)
result_df_Cells.shape



(543, 374)

In [ ]:
#Aggregating values from each Image in Nuclei table

connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Nuclei"
data_frame_objects = pd.read_sql(query, connection)
data_frame_objects.to_csv(f"{inputDir}/Nuclei.csv", index=False) 
data_frame_objects.head()
data_frame_objects.shape
grouped_df = data_frame_objects.groupby('ImageNumber')

# Applying aggregate functions (e.g., sum, mean) to each group
result_df_Nuclei = grouped_df.mean().reset_index(drop=False)
result_df_Nuclei.shape


(543, 374)

In [ ]:
#Aggregating values from each Image in Cytoplasm table

connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Cytoplasm"
data_frame_objects = pd.read_sql(query, connection)
data_frame_objects.to_csv(f"{inputDir}/Cytoplasm.csv", index=False) 

#data_frame.columns

grouped_df = data_frame_objects.groupby('ImageNumber')

# Applying aggregate functions (e.g., sum, mean) to each group
result_df_cytoplasm = grouped_df.mean().reset_index(drop=False)

result_df_cytoplasm.shape


(543, 188)

In [ ]:
#Aggregating values from each Image in Image table

connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Image"
data_frame_image = pd.read_sql(query, connection)

data_frame_image.to_csv(f"{inputDir}/Image.csv", index=False) 
data_frame_image.head()
data_frame_image.shape
#data_frame.columns

(659, 443)

In [ ]:
# Merge the DataFrames Cells, Nuclei, Cytoplasm and Image using ImageNumber as 'ID'
merged_df = pd.merge(result_df_Cells, result_df_Nuclei, on='ImageNumber', how='inner')
merged_df = pd.merge(merged_df, result_df_cytoplasm, on='ImageNumber', how='inner')
merged_df = pd.merge(merged_df, data_frame_image, on='ImageNumber', how='inner')
merged_df.head()
print(merged_df.shape)

(543, 1376)


In [ ]:
# Check and drop for empty columns 

empty_columns = merged_df.columns[merged_df.isnull().all()]
num_empty_columns = len(empty_columns)

# Display the result
if num_empty_columns == 0:
    print("No empty columns.")
else:
    print(f"There are {num_empty_columns} empty columns:")
    print(empty_columns)

# drop empty columns
merged_df_dropped_empty = merged_df.drop(empty_columns, axis=1)
merged_df_dropped_empty.shape

There are 266 empty columns:
Index(['Image_Intensity_LowerQuartileIntensity_Brightfield_Cytoplasm',
       'Image_Intensity_LowerQuartileIntensity_Brightfield_Mtcndr_sgmntd',
       'Image_Intensity_LowerQuartileIntensity_Brightfield_Nuclei',
       'Image_Intensity_LowerQuartileIntensity_Brightfield_Nuclel_sgmntd',
       'Image_Intensity_LowerQuartileIntensity_DNA_Cytoplasm',
       'Image_Intensity_LowerQuartileIntensity_DNA_Mitocondria_segmented',
       'Image_Intensity_LowerQuartileIntensity_DNA_Nuclei',
       'Image_Intensity_LowerQuartileIntensity_DNA_Nucleoli_segmented',
       'Image_Intensity_LowerQuartileIntensity_ER_Cytoplasm',
       'Image_Intensity_LowerQuartileIntensity_ER_Mitocondria_segmented',
       ...
       'Image_Intensity_UpperQuartileIntensity_Mito_Cytoplasm',
       'Image_Intensity_UpperQuartileIntensity_Mito_Mitocondria_segmentd',
       'Image_Intensity_UpperQuartileIntensity_Mito_Nuclei',
       'Image_Intensity_UpperQuartileIntensity_Mito_Nucleoli_segm

In [ ]:
# Rename the first column to include the label Metadara
sorted_df_rename = merged_df_dropped_empty.rename(columns={'Image_ImageNumber': 'Image_Metadata_ImageNumber'})

In [ ]:
#Sort the Metadata columns. The first column should be the ones that contain the metadata.

columns = merged_df_dropped_empty.columns


sorted_columns = sorted(columns, key=lambda x: x.startswith("Image_Metadata"), reverse=True)

# Create a new DataFrame with sorted columns
sorted_df = merged_df_dropped_empty[sorted_columns]
sorted_df.head()

,Image_Metadata_DiseaseGroup,Image_Metadata_Frame,Image_Metadata_Series,Image_Metadata_Site,Image_Metadata_Well,Image_Metadata_WellColumn,Image_Metadata_WellRow,Image_Metadata_Plate,ImageNumber,Cells_Number_Object_Number,...,Image_URL_Membrane,Image_URL_Mito,Image_URL_RNA_nucleoli,Image_Width_Brightfield,Image_Width_DNA,Image_Width_ER,Image_Width_Membrane,Image_Width_Mito,Image_Width_RNA_nucleoli,Metadata_genotype
0,DS2U_DS1 iN,0,0,01,B02,02,02,plate4,1,2.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
1,DS2U_DS1 iN,0,0,02,B02,02,02,plate4,2,5.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
2,DS2U_DS1 iN,0,0,07,B02,02,02,plate4,7,1.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
3,DS2U_DS1 iN,0,0,10,B02,02,02,plate4,10,9.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
4,DS2U_DS1 iN,0,0,12,B02,02,02,plate4,12,1.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U


In [ ]:
#save the dataset sorted 
sorted_df.to_csv(f"{inputDir}/sorted_df.csv", index=False) 

In [ ]:
#delete the Image prefix in the columns.
sorted_df.columns = [col.replace('Image_Metadata', 'Metadata') if 'Image_Metadata' in col else col for col in sorted_df.columns]
sorted_df.to_csv(f"{inputDir}/sorted_df.csv", index=False) 

In [ ]:
# Extract column names with 'Metadata'
metadata_columns = [col for col in sorted_df.columns if 'Metadata' in col]
print(len(metadata_columns))

# Extract column names with 'Cells', 'Nuclei', or 'Cytoplasm'
cell_related_columns = [col for col in sorted_df.columns if any(keyword in col for keyword in ['Cells', 'Nuclei', 'Cytoplasm'])]
print(len(cell_related_columns))



10
1010


In [ ]:

# Extract column names with 'Image' prefix, excluding those already in cell_related_columns
image_columns = [col for col in sorted_df.columns if col.startswith('Image') and col not in cell_related_columns and col not in metadata_columns]


# Determine the remaining columns
other_columns = [col for col in sorted_df.columns if col not in metadata_columns + cell_related_columns + image_columns]


# Concatenate the column lists in the desired order
sorted_columns_final = metadata_columns + other_columns + cell_related_columns + image_columns


# Create a new DataFrame with the sorted columns
sorted_df_final = sorted_df[sorted_columns_final]

In [ ]:
# Explicitly convert all non-metadata columns to float
# in the original df ALL columns are of type object and that breaks pycytominer
for col in sorted_df_final.columns:
    if 'Nuclei' in col or 'Cells' in col or 'Cytoplasm' in col or 'ImageNumber' in col:
        sorted_df_final[col] = sorted_df_final[col].astype(float)
#print([df.dtypes])

sorted_df_final.to_csv(f"{inputDir}/sorted_df_final.csv", index=False)


In [ ]:
#Check if there is come columns duplicated
duplicated_columns = sorted_df_final[sorted_df_final.duplicated()]

if len(duplicated_columns) > 0:
    print(f"Duplicate columns found: {', '.join(duplicated_columns)}")
else:
    print("No duplicate columns found.")

No duplicate columns found.


Some columns have two objects contained in the name, as nuclei and cytoplasm, which bring error because the code expect to find columns with Nuclei, Cells OR Cytoplasm, not two of them together. Then, we define a different name for the suffix to fix it.

In [ ]:
# Define the substrings to replace
substrings_to_replace = ['_Nuclei']
replacement_substrings = ['_Nucle']

# Replace substrings in column names
sorted_df_final.columns = [col.replace(old, new) for col in sorted_df_final.columns for old, new in zip(substrings_to_replace, replacement_substrings)]


In [ ]:
# Define the substrings to replace
substrings_to_replace = ['_Cells']
replacement_substrings = ['_Cell']

# Replace substrings in column names
sorted_df_final.columns = [col.replace(old, new) for col in sorted_df_final.columns for old, new in zip(substrings_to_replace, replacement_substrings)]


In [ ]:
# Define the substrings to replace
substrings_to_replace = ['_Cytoplasm']
replacement_substrings = ['_Cytoplas']

# Replace substrings in column names
sorted_df_final.columns = [col.replace(old, new) for col in sorted_df_final.columns for old, new in zip(substrings_to_replace, replacement_substrings)]


In [ ]:
# Filter columns that contain "Nuclei," "Cytoplasm," and "Cells" and concatenate in a dataframe
filtered_df = sorted_df_final.filter(like='Nuclei').join(sorted_df_final.filter(like='Cytoplasm')).join(sorted_df_final.filter(like='Cells'))
filtered_df.to_csv(f"{inputDir}/filtered_df.csv", index=False)

In [ ]:


# Get column names that start with "Metadata_"
metadata_column_names = [col for col in sorted_df_final.columns if col.startswith('Metadata_')]

# Create a new DataFrame with the filtered columns
Columns_metadata = sorted_df_final[metadata_column_names]
Columns_metadata.to_csv(f"{inputDir}/Columns_metadata.csv", index=False)


In [ ]:
# Define the aggregation functions
#This will aggregate by Well all the features measurments that come from the same Well defined in Metadata_Well column
aggregations = {
    'Metadata_Well': 'first',  # Take the first value as it's the same for each group
    **{col: ['mean', 'median', 'std', 'min', 'max'] for col in filtered_df.columns}
}
# Group by "Metadata_ImageNumber" and aggregate the statistics for columns with the specified prefix
# The .agg (aggregate) method applies the listed operations to all values grouped by ImageNumber
well_agg_df = sorted_df_final.groupby(['Metadata_Well','Metadata_genotype'], sort=False).agg(aggregations)

In [ ]:
# Flatten MultiIndex columns
well_agg_df.columns = well_agg_df.columns.map('_'.join)

# Reset index to move 'Metadata_ImageNumber' to a regular column
well_agg_df.reset_index(inplace=True)

/var/folders/mm/mp46tk592c10r7w5n0vwcclc0000gq/T/ipykernel_64893/292143542.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  well_agg_df.reset_index(inplace=True)
/var/folders/mm/mp46tk592c10r7w5n0vwcclc0000gq/T/ipykernel_64893/292143542.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  well_agg_df.reset_index(inplace=True)


In [ ]:
well_agg_df.head()
well_agg_df.to_csv(f"{inputDir}/well_agg_df.csv", index=False)

In [ ]:
well_agg_df = pd.read_csv("/Users/llanos/Documents/GitHub/Kyra_project/Batch3/well_agg_df.csv")

One we got our features aggregated by Well we can normalize. The neuron dataset was normalized based on the negative control assigned to each group, this example is done for the Grpup WC, and the control used was WCB

In [ ]:
pycytominer.normalize(
    profiles=well_agg_df,    #the dataframe containing the CellProfiler features
    #features=feature_cols,   #list of features to normalize; if "infer", anything starting with 'Cells', 'Nuclei' or 'Cytoplasm'
    #meta_features=metadata_cols,   #list of metadata columns; if "infer" (default), any column starting with 'Metadata_'
    image_features=False,  #Whether the profiles contain image features
    samples="Metadata_genotype=='WCB'",     #The metadata column values to use as a normalization reference. Defaults to "all".
    method="mad_robustize",    #How to normalize the dataframe. Defaults to "standardize". "mad_robustize" is used in the profiling recipe by default
    mad_robustize_epsilon=0, # The mad_robustize fudge factor parameter. Set this to 0 if mad_robustize generates features with large values.
    output_file=f"{inputDir}/data_normalized_WC.csv" 
)
df_norm = pd.read_csv(f"{inputDir}/data_normalized_WC.csv")
df_norm.shape

(20, 4663)

This step should be run if you already generate 2 dataframes for each group, DS and WC. Once you got those 2 dataframe or csv you can do the feature selection.
CONCATENATE BEFORE FUTURE SELECT:



In [ ]:
df_norm_DS = pd.read_csv(f"{inputDir}/data_normalized.csv")


In [ ]:
df_norm_DS.shape #checking the shape of the DS group

(20, 4663)

In [ ]:
df_norm.shape #checking the shape of the WC group

(20, 4663)

In [ ]:
#Concatenate plates

df_norm_WC_DS = pd.concat([df_norm, df_norm_DS], ignore_index=True)

In [ ]:
#checking the new shape
df_norm_WC_DS.shape

(40, 4663)

In [ ]:
pycytominer.feature_select(
    profiles=df_norm_WC_DS,
    #features=feature_cols,   #list of features to normalize; if "infer", anything starting with 'Cells', 'Nuclei' or 'Cytoplasm'
    image_features=False,  #Whether the profiles contain image features
    operation=['variance_threshold','correlation_threshold', 'drop_na_columns', 'blocklist', 'drop_outliers'],   #add outlier removal to get rid of 
    outlier_cutoff=500, #500 is the default
    output_file=f"{inputDir}/data_normalized_featureSelected_WC_DS_bygenotype.csv", 
    samples='all', #Samples to provide operation on.Str or list. Defaults to 'all' ...I don't get it
)
df_ftsel_WC_DS = pd.read_csv(f"{inputDir}/data_normalized_featureSelected_WC_DS_bygenotype.csv")
df_ftsel_WC_DS.shape

(40, 1439)

In [ ]:
#Check the shape and save the future selected file
df_ftsel_WC_DS.shape
df_ftsel_WC_DS.to_csv(f"{inputDir}/df_ftsel_WC_DS.csv", index=False)


In [ ]:
# Reorder and transpose df for easier import into Morpheus
df_ftsel_WC_DS = df_ftsel_WC_DS.T

#save to final .csv
# Index=True is necessary so that we save the row labels into the .csv
df_ftsel_WC_DS.to_csv(f"{inputDir}/data_forMorpheus_WC_DS_by_genotype.csv",sep=',',encoding="utf-8",header=False,index=True)